# Game basics and problem representation

## Game description

## Project goals

how this problem could be analyzed, find different options

## Name the game concepts

We need to name all the concepts. This will make it easier to create classes and functions later
- the goal is to fill a board
- the board consist of cells
- the board organize cells horizontally in rows 
- the board organize cells vertically in columns (shorten as cols) 
- the cells have states which are undefined, solid/black and filler
- game input is given as a list clues of for each row or col  
- clue is either a number or a list of numbers 
- when there is only one block of black cells in the row or column the number gives the size of the block of contiguous cells
- where there are multiple blocks of black cells in the row or col the clue is a list of sizes
- the goal of the game is to find where blacks and fillers are so that the rules given by the clues holds

## How to represent the clues 

You can stop and think for yourself how you would represent the clues to represent what is defined above

### wait

### clues

There are many options. 
My choice is adict of two lists. Each list may contain numbers or list of numbers. I made my decision based on what is easier to use when writing instructions. 

Please note that the list will need to be handled carefully and integers will have to converted to a list to use the liust elements in a consistent way

In [1]:
# game instructions for a 5x5 game

# Please notee that list are base 0 so clues['cols'][1] is col 2
# 4 denotes that col 2 has 4 contiguous black cells 
#    and a filler of 1 which position is unknown
# [2,2] denotes that col 3 has 2 contiguous black cells, 
#    a filler which size is unknows and 2 contiguous black cels

clues = {
    'rows': [1, 3, [1,1], 3, 5],
    'cols': [1, 4, [2,2], 4, 1]
}

### board

There are different ways to represent a board
- 2D matrix : a list (rows) of lists (columns)
- 1D : a list of cells, the position of the cell is computed from the row and col number
- Network : a network a cells linked to each other horizontally and vertically

The goal is to solve the board and print it, so I will go for the most straightforward

Numpy provides an efficient way of storing matrix and in addition allows to change the shape, for instance to witch between two dimensions and one when needed.

We will need some convention to store the state of the cells as undefined, black or filler
- undefined : -1
- filler : 0
- black/solid : 1

The rationale is that the sum of blacks should be easy to compare with the ckue value. So black should be 1. It make sense that fillers are 0 so they do not later the sums. Undefined could be anything we chan filter out easily.

We will also need to represent the board in order to check what happen.
A simple representation of the board to start with be to convert this numbers into characters
- undefined : .
- filler : x
- black/solid : o

I make the assumption that the board can be represented as a plot with matplotlib. We will check this later.

### MVP board 2x2

We will build a mini board of 4 cells, create some function and try to use them while solving the game manually

In [2]:
import numpy as np

In [3]:
# clues for mini game
clues = {
    'rows': [1, 2],
    'cols': [2, 1]
}

# solution for the mini game
#  ox
#  oo

# create a 2x2 board of type int, initialized with -1
states = np.full((2, 2), -1, dtype=int)
states

array([[-1, -1],
       [-1, -1]])

In [4]:
# give the solution to the game
# index is row, cell
states[0,0]=1
states[0,1]=0
states[1,0]=1
states[1,1]=1
states

array([[1, 0],
       [1, 1]])

In [5]:
f_marks = lambda v: v*2
marks = f_marks(states)
marks

array([[2, 0],
       [2, 2]])

In [6]:
marks_switcher = {-1: '.', 0: 'x', 1: 'o'}
print(marks_switcher[-1])
print(marks_switcher[0])
print(marks_switcher[1])


.
x
o


In [7]:
marks_switcher = {-1: '.', 0: 'x', 1: 'o'}
# for whatever reason lambda does not work here
def f_marks(v):
    return marks_switcher[v]
applyall = np.vectorize(f_marks)
marks = applyall(states)
marks

array([['o', 'x'],
       ['o', 'o']], dtype='<U1')

### class implementation

In [8]:
# reset variables from tests
%reset -f

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [9]:
import numpy as np
from enum import Enum

# constants for mark lette
class BoardMark(Enum):
    INIT = -1
    FILLER = 0
    BLACK = 1

class Board:
    '''
    Manage the board.
    '''
    
    def __init__(self, some_clues):
        '''
        board constructor
        '''
        # given parameters
        self.clues = some_clues
        # compute board dimensions
        self.width = len(self.clues['cols']) 
        self.height = len(self.clues['rows']) 
        # create board of type int, initialized with -1
        default_value = BoardMark.INIT.value
        self.states = np.full((self.height, self.width), default_value, dtype=int)
        
    def mark(self, row, col, mark):
        '''
        mark a cell
        '''
        self.states[row, col] = mark.value

    def prettyprint(self):
        '''
        pretty print of the board 
        '''
        print("cols:", end=" ")
        print(*self.clues['cols']) # * unpacks and remove []s
        print("rows:")
        [print(r) for r in self.clues['rows']]
          
        marks_switcher = {-1: '.', 0: 'x', 1: 'o'}
        # for whatever reason lambda does not work here
        def f_marks(v):
            return marks_switcher[v]
        applyall = np.vectorize(f_marks)
        marks = applyall(self.states)
        print(marks)


### check mini game

In [10]:
# clues for mini game
clues1 = {
    'rows': [1, 2],
    'cols': [2, 1]
}
board1 = Board(clues1)
board1.prettyprint()

cols: 2 1
rows:
1
2
[['.' '.']
 ['.' '.']]


In [11]:
# give the solution to the game
# index is row, cell
board1.mark(0, 0, BoardMark.BLACK) 
board1.mark(0, 1, BoardMark.FILLER) 
board1.mark(1, 0, BoardMark.BLACK) 
board1.mark(1, 1, BoardMark.BLACK) 

board1.prettyprint()

cols: 2 1
rows:
1
2
[['o' 'x']
 ['o' 'o']]


### check if not square 4x2

In [13]:
# clues for mini game
clues1 = {
    'rows': [1, 2, 1, 2],
    'cols': [2, 1]
}
board1 = Board(clues1)
board1.prettyprint()

cols: 2 1
rows:
1
2
1
2
[['.' '.']
 ['.' '.']
 ['.' '.']
 ['.' '.']]


# TODO nb undefined, compute clues from board